In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import mysql.connector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


conn = mysql.connector.connect(
    host = "roseda.cj6s66ayq6e8.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "fo",
    password = "****",
    database = "folium"
)

cursor = conn.cursor()

driver = webdriver.Chrome(service=Service("/home/hyun/dev_ws/EDA/driver/chromedriver"))


def extract_and_insert_data(a_tag, selected_year):
    # onclick 속성의 JavaScript 함수를 가져옴 (예: popup('1701999933'))
    onclick_function = a_tag['onclick']

    # JavaScript 함수를 실행
    driver.execute_script(onclick_function)

    # 필요하다면 클릭 후 잠시 대기
    driver.implicitly_wait(1)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    #td_mbpd 클래스 기업명, 설립연도, 종업원 수, 투자유치금액, 현재라운드/최근 투자유치 연도 얻을 수 있다.
    info = soup.find_all(class_="td_mbpd")
    text_info = [item.get_text(strip=True) for item in info]

    #기업명
    company_name = text_info[0]

    #종업원 수
    employee_count = text_info[3] if text_info[3] else None

    #투자유치금액
    investment_amount = text_info[4] if text_info[4] else None

    #현재라운드 / 최근 투자 유치 연도
    if text_info[5] and '/' in text_info[5]:
        round = text_info[5].split('/')[0].strip() if text_info[5].split('/')[0] else None
        recent_funding_year = text_info[5].split('/')[1].strip() if text_info[5].split('/')[1] else None
    else:
        round = None
        recent_funding_year = None

    #주소
    address = soup.find_all("td", class_="view_wt")
    address = [item.get_text(strip=True) for item in address]
    address = address[2] if address and address[2] else None

    #인재상
    ideal_talent = soup.find("td", class_="aicp_view view_wt")
    ideal_talent = ideal_talent.get_text(strip=True) if ideal_talent else None

    sql = "INSERT INTO company_info (company_name, employee_count, investment_amount, round, recent_funding_year, address, ideal_talent, selected_year) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values = (company_name, employee_count, investment_amount, round, recent_funding_year, address, ideal_talent, selected_year)
    cursor.execute(sql, values)
    conn.commit()    

    #X 버튼 클릭
    close_button = driver.find_element(By.ID, "close")
    driver.execute_script("arguments[0].click();", close_button)


def scrape_and_insert_data(type, prefix, id_value, industry_type, cursor, conn):
    url = f"https://www.ailandscape.net/product/list.php?type={type}&ca_id={id_value}"
    driver.get(url)

    driver.implicitly_wait(1)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    strong_tags = soup.find_all('strong')
    ul_tags = soup.find_all('ul')

    category = None
    for tag in ul_tags:
        p_tag = tag.find('p')
        if p_tag:
            category = p_tag.get_text(strip=True)

    for tag in tqdm(strong_tags):
        a_tag = tag.find('a')
        name = tag.get_text(strip=True)

        homepage = a_tag['href'] if a_tag and 'href' in a_tag.attrs else None

        sql = "INSERT INTO company_info2 (company_name, category, homepage, industry_type) VALUES (%s, %s, %s, %s)"
        values = (name, category, homepage, industry_type)
        cursor.execute(sql, values)
        conn.commit()


for type in range(1, 3):  

    industry_type = "industry" if type == 1 else "cross-industry"
  
    for prefix in range(4, 0, -1):  # 4부터 1까지 감소
        # 4010부터 4090까지 및 40a0, 40b0 등으로 변경
        selected_year = int(f"202{prefix}")

        for i in range(1, 10):  # 4010에서 4090까지
            id_value = f"{prefix}0{i}0"  # '4010', '4020', ..., '4090'
            scrape_and_insert_data(type, prefix, id_value, industry_type, cursor, conn)

            url = f"https://www.ailandscape.net/product/list.php?type={type}&ca_id={id_value}"
            driver.get(url)
                        
            driver.implicitly_wait(1)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            

            a_tags = soup.find_all('a', onclick=True)

            for a_tag in tqdm(a_tags):
                extract_and_insert_data(a_tag, selected_year)

            driver.implicitly_wait(1)



        for char in range(ord('a'), ord('z') + 1):  # 'a'부터 'z'까지
            id_value = f"{prefix}0{chr(char)}0"  # '40a0', '40b0'
            scrape_and_insert_data(type, prefix, id_value, industry_type, cursor, conn)            
            url = f"https://www.ailandscape.net/product/list.php?type={type}&ca_id={id_value}"
            driver.get(url)
            

            driver.implicitly_wait(1)

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            

            a_tags = soup.find_all('a', onclick=True)

            for a_tag in tqdm(a_tags):
                extract_and_insert_data(a_tag, selected_year)

            driver.implicitly_wait(1)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]